## Harsh Bansal
## B.tech CSE


# Importing Modules and Datasets

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.datasets import boston_housing

In [2]:
(train_x,train_y),(test_x,test_y) = boston_housing.load_data()


65536/57026 [==================================] - 0s 0us/step


In [3]:
features = ['CRIM', 'ZN','INDUS','CHAS','NOX','RM','AGE','DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

# Converting to pandas DataFrame

In [4]:
df_train_x = pd.DataFrame(train_x,columns=features)
df_test_x = pd.DataFrame(test_x,columns=features)
df_train_y = pd.DataFrame(train_y,columns =['MEDV'])
df_test_y = pd.DataFrame(test_y,columns =['MEDV'])


In [5]:
df_train_x.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


# Normalization

In [6]:
mean = df_train_x.mean(axis=0)
std = df_train_x.std(axis=0)
df_train_x-=mean
df_train_x/=std

In [7]:
df_test_x-=mean
df_test_x/=std

# Feature Columns

In [8]:
feature_columns_numeric = []
feature_columns_numeric = [tf.feature_column.numeric_column(fname, dtype=tf.float32) for fname in features]

# Data Pipeline

In [9]:
def estimator_input_fn(df_data,df_label,epochs=10,shuffle=True,batch_size=32):
    def input_fn():
        ds= tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_fn

In [10]:
train_input_fn = estimator_input_fn(df_train_x,df_train_y)
val_input_fn = estimator_input_fn(df_test_x, df_test_y, epochs=1, shuffle=False)

# Building Model

In [11]:
model = tf.estimator.LinearRegressor(feature_columns = feature_columns_numeric , optimizer="RMSProp")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvyagxfmz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
model.train(train_input_fn,steps=100)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpvyagxfmz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 743.96533, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpvyagxfmz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 19.894264.


In [13]:
result = model.evaluate(val_input_fn)
print(result)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-24T12:03:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvyagxfmz/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.69052s
INFO:tensorflow:Finished evaluation at 2022-01-24-12:03:53
INFO:tensorflow:Saving dict for global step 100: average_loss = 31.607178, global_step = 100, label/mean = 23.078432, loss = 36.64384, prediction/mean = 19.572704
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmpvyagxfmz/model.ckpt-100
{'average_loss': 31.607178, 'label/mean': 23.078432, 'loss': 36.64384, 'prediction/mean': 19.572704, 'global_step': 100}


# Predicting Result

In [14]:
result = model.predict(val_input_fn)  

In [15]:

for pred,exp in zip(result, test_y[:32]): 
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvyagxfmz/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value:  5.6531067 Expected:  7.2
Predicted Value:  17.885578 Expected:  18.8
Predicted Value:  17.691849 Expected:  19.0
Predicted Value:  28.66755 Expected:  27.0
Predicted Value:  22.634993 Expected:  22.2
Predicted Value:  17.546944 Expected:  24.5
Predicted Value:  26.095894 Expected:  31.2
Predicted Value:  21.911427 Expected:  22.9
Predicted Value:  14.68394 Expected:  20.5
Predicted Value:  18.130972 Expected:  23.2
Predicted Value:  17.621122 Expected:  18.6
Predicted Value:  15.017524 Expected:  14.5
Predicted Value:  12.766444 Expected:  17.8
Predicted Value:  32.37277 Expected:  50.0
Predicted Value:  14.335949 Expected:  20.8
Predicted Value:  17.03204 Expected:  24.3
Predicted Value:  21.773064 Expected:  24.2
Predicted Value:  20.083